In [2]:
# import libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler

# 0. Read data

In [3]:
df = pd.read_csv('../data/spotify_clean_reg.csv')
df_outliers = pd.read_csv('../data/spotify_clean_reg_without_outliers.csv')

In [4]:
df.head()

,Unnamed: 0,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,duration_s
0,0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814,99.373
1,1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816,137.373
2,2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368,170.267
3,3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227,152.427
4,4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390,82.625


In [5]:
df = df.drop(columns=['Unnamed: 0'])
df_outliers = df_outliers.drop(columns=['Unnamed: 0'])

### 1. Drop the useless columns

In [7]:
df_1 = df.drop(columns=['genre', 'artist_name', 'track_name', 'track_id'])
df_out_1 = df_outliers.drop(columns=['genre', 'artist_name', 'track_name', 'track_id'])

### 2. Transform to dummy

In [8]:
df_trans = pd.get_dummies(df_1, columns=['key','time_signature', 'mode'])
df_trans_out = pd.get_dummies(df_out_1, columns=['key','time_signature', 'mode'])

### 3. Perform Train Test Split

In [9]:
features = df_trans.drop(columns=["popularity"], axis = 1)
target = df_trans["popularity"]
features_out = df_trans_out.drop(columns=["popularity"], axis = 1)
target_out = df_trans_out["popularity"]

In [10]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

In [11]:
X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(features_out, target_out, test_size=0.20, random_state=0)

### 4. Model Selection

#### 4.1 k-NN

In [19]:
number_of_neighbours = 80

In [20]:
knn = KNeighborsRegressor(n_neighbors=number_of_neighbours)

In [21]:
knn.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=80)

In [22]:
knn.score(X_test, y_test)

0.22476770236940358

In [18]:
l = []
for i in range(10,50):
    
    knn = KNeighborsRegressor(n_neighbors=i)
    knn.fit(X_train, y_train)
    r2 = knn.score(X_test, y_test)
    l.append(r2)
    
    
    print(f"k = {i}  || R^2 = {r2}")

k = 10  || R^2 = 0.2128744779328272
k = 11  || R^2 = 0.21337695536840529
k = 12  || R^2 = 0.2145918920504829
k = 13  || R^2 = 0.2168771089351631
k = 14  || R^2 = 0.21767053918943102


KeyboardInterrupt: 